In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Aug  1 05:59:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
print("============查看cpu配置================")
# 查看cpu配置
!cat /proc/cpuinfo | grep model\ name
print("=============查看内存容量===============")
# 查看内存容量
!cat /proc/meminfo | grep MemTotal

============查看cpu配置================
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
=============查看内存容量===============
MemTotal:       13298580 kB


In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat Jul 30 15:02:44 2022

@author: lvbinghan
"""

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat Jul  9 21:43:53 2022

@author: lvbinghan
"""

# importing system libraries

from os import walk
from string import punctuation
from random import shuffle
from collections import Counter
from sklearn.model_selection import cross_val_predict
# importing additional libraries

import pandas as pd
import sklearn as sk
import nltk
import pytest
nltk.download('stopwords')
import time
start_time = time.time()



from nltk.test.classify_fixt import setup_module
setup_module()
from os import walk
from string import punctuation
from random import shuffle
from collections import Counter
from sklearn.model_selection import cross_val_predict
# importing additional libraries

import pandas as pd
import sklearn as sk
import nltk
import pytest




from nltk.test.classify_fixt import setup_module
setup_module()

import pandas as pd
import numpy as np
import re
import collections

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import warnings
warnings.simplefilter(action='ignore', category=Warning)
import keras
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pickle





# Read the whole data from the Enron Dataset into a variable allData.

pathwalk = walk(r"/content/gdrive/MyDrive/enron1/")

allHamData, allSpamData = [], []
for root, dr, file in pathwalk:
    if 'ham' in str(file):
        for obj in file:
            with open(root + '/' + obj, encoding='latin1') as ip:
                allHamData.append(" ".join(ip.readlines()))
                
    elif 'spam' in str(file):
        for obj in file:
            with open(root + '/' + obj, encoding='latin1') as ip:
                allSpamData.append(" ".join(ip.readlines()))
                
                
                

allHamData = list(set(allHamData))
allSpamData = list(set(allSpamData))


hamPlusSpamData = allHamData + allSpamData
labels = ["ham"]*len(allHamData) + ["spam"]*len(allSpamData)

raw_df = pd.DataFrame({"email": hamPlusSpamData, 
                       "label": labels})



raw_df.sample(5)
raw_df['email']

""""""""""""""""""""""""""""""""""""""""""


def preprocess(data):
    # tokenization
    tokens = nltk.word_tokenize(data)
    tokens = [w.lower() for w in tokens if w.isalpha()]

    # finding uncommon words
    cnt = Counter(tokens)
    uncommons = cnt.most_common()[:-int(len(cnt)*0.1):-1]
    
    # listing stopwords from NLTK
    stops = set(nltk.corpus.stopwords.words('english'))

    # removing stop words and uncommon words
    tokens = [w for w in tokens if (w not in stops and w not in uncommons)]

    # lemmatization
    lemmatizer = nltk.WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(w, pos='a') for w in tokens]

    return tokens

label_encoder = sk.preprocessing.LabelEncoder()
raw_df['label'] = label_encoder.fit_transform(raw_df.label)
import string
from nltk.corpus import stopwords

def process_text(text):
    no_punc = [char for char in text if char not in string.punctuation]
    no_punc = ''.join(no_punc)
    return ' '.join([word for word in no_punc.split() if word.lower() not in stopwords.words('english')])

raw_df['email']=raw_df['email'].apply(process_text)
raw_df['email']

from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
def stemming (text):
    return ''.join([stemmer.stem(word) for word in text])
raw_df['email']=raw_df['email'].apply(stemming)
raw_df.head()


from sklearn.feature_extraction.text import CountVectorizer
vectorizer= CountVectorizer()
message_bow = vectorizer.fit_transform(raw_df['email'])

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(message_bow,raw_df['label'],test_size=0.25)

X = message_bow
y = raw_df['label']




# TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(raw_df['email'])

# Transform entire BoW into tf-idf corpus
message_bow_tfidf = tfidf_transformer.transform(raw_df['email'])

X_tfidf = message_bow_tfidf
y = raw_df['label']
X_tfidf_train,X_tfidf_test,y_tfidf_train,y_tfidf_test = train_test_split(message_bow_tfidf,raw_df['label'],test_size=0.25)

label_encoder = sk.preprocessing.LabelEncoder()
y_tfidf = label_encoder.fit_transform(raw_df.label)



# ANN data processing
X = message_bow
y = raw_df['label']



from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)


X_train_sc = sc.fit(X_train)
X_train = X_train_sc.transform(X_train)


X_train_ann = X_train.copy()
X_train_ann = X_train_ann.toarray()

X_ann = X.copy()
X_ann = X_ann.toarray()







[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


ValueError: ignored

In [ ]:

#数据预处理测试



print("this is naive_bayes part")
print("=======================================================================================================")
from sklearn.naive_bayes import MultinomialNB
nb= MultinomialNB()
nb.fit(X_train,y_train)
y_pred = (nb.predict(X_test) > 0.5).astype("int32")
from sklearn.metrics import classification_report
print("nb score",classification_report(y_test,y_pred, digits=4))


from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

from sklearn.model_selection import cross_val_score
#nb2= MultinomialNB()
#scores_nb2 = cross_val_score(nb2, X, y, cv=10)
#print("nb corss validation :  %0.4f accuracy with a standard deviation of %0.4f" % (scores_nb2.mean(), scores_nb2.std()))
#y_pred_nb = (nb.predict(X_test) > 0.5).astype("int32")
#y_pred_nb_cv = cross_val_predict(nb2, X, y, cv=10)
#print(classification_report(y_pred_nb_cv,y, digits=4))

nb3= MultinomialNB()
accuracies_nb_accuracy = cross_val_score(estimator=nb3, scoring="accuracy", 
    X=X_tfidf, y=y_tfidf, cv=10)
accuracies_nb_f1 = cross_val_score(estimator=nb3, scoring="f1", 
    X=X_tfidf, y=y_tfidf, cv=10)
accuracies_nb_recall = cross_val_score(estimator=nb3, scoring="recall", 
    X=X_tfidf, y=y_tfidf, cv=10)
accuracies_nb_precision = cross_val_score(estimator=nb3, scoring="precision", 
    X=X_tfidf, y=y_tfidf, cv=10)

print("NB corss validation :  %0.4f accuracy  with a standard deviation of %0.4f" % (accuracies_nb_accuracy.mean(), accuracies_nb_accuracy.std()))
print("NB corss validation :  %0.4f f1     with a standard deviation of %0.4f" % (accuracies_nb_f1.mean(), accuracies_nb_f1.std()))
print("NB corss validation :  %0.4f recall   with a standard deviation of %0.4f" % (accuracies_nb_recall.mean(), accuracies_nb_recall.std()))
print("NB corss validation :  %0.4f precision with a standard deviation of %0.4f" % (accuracies_nb_precision.mean(), accuracies_nb_precision.std()))



print("this is svm part")
print("=======================================================================================================")

from sklearn import svm
#svmc= svm.SVC()
#svmc.fit(X_tfidf_train,y_tfidf_train)
#y_pred = svmc.predict(X_tfidf_test)
#from sklearn.metrics import classification_report
#print("svmc score",classification_report(y_tfidf_test,y_pred, digits=4))
from sklearn.metrics import confusion_matrix
#confusion_matrix(y_tfidf_test, y_pred)
from sklearn.model_selection import cross_val_score
#svmc2= svm.SVC()
#scores_svm = cross_val_score(svmc2, X_tfidf, y_tfidf, cv=10)
#print("svm corss validation :  %0.4f accuracy with a standard deviation of %0.4f" % (scores_svm.mean(), scores_svm.std()))
#y_pred_svm = (svmc.predict(X_test) > 0.5).astype("int32")
#y_pred_svm_cv = cross_val_predict(svmc2, X_tfidf, y_tfidf, cv=10)
#print(classification_report(y_pred_svm_cv,y, digits=4))

svmc3= svm.SVC()
accuracies_svmc_accuracy = cross_val_score(estimator=svmc3, scoring="accuracy", 
    X=X_tfidf, y=y, cv=10)
accuracies_svmc_f1 = cross_val_score(estimator=svmc3, scoring="f1", 
    X=X_tfidf, y=y, cv=10)
accuracies_svmc_recall = cross_val_score(estimator=svmc3, scoring="recall", 
    X=X_tfidf, y=y, cv=10)
accuracies_svmc_precision = cross_val_score(estimator=svmc3, scoring="precision", 
    X=X_tfidf, y=y, cv=10)

print("SVM corss validation :  %0.4f accuracy  with a standard deviation of %0.4f" % (accuracies_svmc_accuracy.mean(), accuracies_svmc_accuracy.std()))
print("SVM corss validation :  %0.4f f1     with a standard deviation of %0.4f" % (accuracies_svmc_f1.mean(), accuracies_svmc_f1.std()))
print("SVM corss validation :  %0.4f recall   with a standard deviation of %0.4f" % (accuracies_svmc_recall.mean(), accuracies_svmc_recall.std()))
print("SVM corss validation :  %0.4f precision with a standard deviation of %0.4f" % (accuracies_svmc_precision.mean(), accuracies_svmc_precision.std()))




print("this is r-tree part")
print("=======================================================================================================")
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification


R_tree = RandomForestClassifier(n_estimators = 1000, min_samples_split = 10, max_depth=100, random_state = 42)
R_tree.fit(X_tfidf_train, y_tfidf_train)



y_pred = R_tree.predict(X_tfidf_test)
from sklearn.metrics import classification_report
print("R_tree score",classification_report(y_tfidf_test,y_pred, digits=4))


from sklearn.metrics import confusion_matrix
#confusion_matrix(y_test, y_pred)

from sklearn.model_selection import cross_val_score
#R_tree2 = RandomForestClassifier(n_estimators = 1000, min_samples_split = 10, max_depth=100, random_state = 42)
#scores_clf_model = cross_val_score(R_tree2, X, y_tfidf, cv=10)
#print("R_tree corss validation :  %0.4f accuracy with a standard deviation of %0.4f" % (scores_clf_model.mean(), scores_clf_model.std()))
#y_pred_tree = (R_tree.predict(X_test) > 0.5).astype("int32")
#y_pred_tree_cv = cross_val_predict(R_tree2, X, y_tfidf, cv=10)
#print(classification_report(y_pred_tree_cv,y, digits=4))

R_tree3 = RandomForestClassifier(n_estimators = 1000, min_samples_split = 10, max_depth=100, random_state = 42)
accuracies_rtree_accuracy = cross_val_score(estimator=R_tree3, scoring="accuracy", 
    X=X_tfidf, y=y, cv=10)
accuracies_rtree_f1 = cross_val_score(estimator=R_tree3, scoring="f1", 
    X=X_tfidf, y=y, cv=10)
accuracies_rtree_recall = cross_val_score(estimator=R_tree3, scoring="recall", 
    X=X_tfidf, y=y, cv=10)
accuracies_rtree_precision = cross_val_score(estimator=R_tree3, scoring="precision", 
    X=X_tfidf, y=y, cv=10)

print("RF corss validation :  %0.4f accuracy  with a standard deviation of %0.4f" % (accuracies_rtree_accuracy.mean(), accuracies_rtree_accuracy.std()))
print("RF corss validation :  %0.4f f1     with a standard deviation of %0.4f" % (accuracies_rtree_f1.mean(), accuracies_rtree_f1.std()))
print("RF corss validation :  %0.4f recall   with a standard deviation of %0.4f" % (accuracies_rtree_recall.mean(), accuracies_rtree_recall.std()))
print("RF corss validation :  %0.4f precision with a standard deviation of %0.4f" % (accuracies_rtree_precision.mean(), accuracies_rtree_precision.std()))




In [ ]:
#测试voting esemble 方法

nb3= MultinomialNB()
R_tree3 = RandomForestClassifier(n_estimators = 1000, min_samples_split = 10, max_depth=100, random_state = 42)
svmc3= svm.SVC()

from sklearn.ensemble import VotingClassifier
eclf = VotingClassifier(estimators=[('nb2', nb3), ('svm', svmc3), ('rtree', R_tree3)], voting='hard')
eclf = eclf.fit(X_tfidf, y)
y_pred = eclf.predict(X)
from sklearn.metrics import classification_report
print("eclf score",classification_report(y,y_pred, digits=4))

accuracies_ann_accuracy = cross_val_score(estimator=eclf, scoring="accuracy", 
    X=X_tfidf, y=y, cv=10)
accuracies_ann_f1 = cross_val_score(estimator=eclf, scoring="f1", 
    X=X_tfidf, y=y, cv=10)
accuracies_ann_recall = cross_val_score(estimator=eclf, scoring="recall", 
    X=X_tfidf, y=y, cv=10)
accuracies_ann_precision = cross_val_score(estimator=eclf, scoring="precision", 
    X=X_tfidf, y=y, cv=10)

print("eclf corss validation :  %0.4f accuracy  with a standard deviation of %0.4f" % (accuracies_ann_accuracy.mean(), accuracies_ann_accuracy.std()))
print("eclf corss validation :  %0.4f f1     with a standard deviation of %0.4f" % (accuracies_ann_f1.mean(), accuracies_ann_f1.std()))
print("eclf corss validation :  %0.4f recall   with a standard deviation of %0.4f" % (accuracies_ann_recall.mean(), accuracies_ann_recall.std()))
print("eclf corss validation :  %0.4f precision with a standard deviation of %0.4f" % (accuracies_ann_precision.mean(), accuracies_ann_precision.std()))



eclf score               precision    recall  f1-score   support

           0     0.9997    0.9946    0.9972      3531
           1     0.9872    0.9993    0.9932      1463

    accuracy                         0.9960      4994
   macro avg     0.9934    0.9970    0.9952      4994
weighted avg     0.9960    0.9960    0.9960      4994

eclf corss validation :  0.9870 accuracy  with a standard deviation of 0.0056
eclf corss validation :  0.9779 f1     with a standard deviation of 0.0095
eclf corss validation :  0.9836 recall   with a standard deviation of 0.0134
eclf corss validation :  0.9725 precision with a standard deviation of 0.0154


In [ ]:
#集成学习测试2

nb3= MultinomialNB()
R_tree3 = RandomForestClassifier(n_estimators = 1000, min_samples_split = 10, max_depth=100, random_state = 42)
svmc3= svm.SVC()

from sklearn.ensemble import VotingClassifier
eclf = VotingClassifier(estimators=[('nb2', nb3), ('svm', svmc3), ('rtree', R_tree3)], voting='hard')
eclf = eclf.fit(X_tfidf_train,y_tfidf_train)
y_pred = eclf.predict(X_tfidf_test)
from sklearn.metrics import classification_report
print("eclf score",classification_report(y_tfidf_test,y_pred, digits=4))

accuracies_ann_accuracy = cross_val_score(estimator=eclf, scoring="accuracy", 
    X=X_tfidf, y=y, cv=10)
accuracies_ann_f1 = cross_val_score(estimator=eclf, scoring="f1", 
    X=X_tfidf, y=y, cv=10)
accuracies_ann_recall = cross_val_score(estimator=eclf, scoring="recall", 
    X=X_tfidf, y=y, cv=10)
accuracies_ann_precision = cross_val_score(estimator=eclf, scoring="precision", 
    X=X_tfidf, y=y, cv=10)

print("eclf corss validation :  %0.4f accuracy  with a standard deviation of %0.4f" % (accuracies_ann_accuracy.mean(), accuracies_ann_accuracy.std()))
print("eclf corss validation :  %0.4f f1     with a standard deviation of %0.4f" % (accuracies_ann_f1.mean(), accuracies_ann_f1.std()))
print("eclf corss validation :  %0.4f recall   with a standard deviation of %0.4f" % (accuracies_ann_recall.mean(), accuracies_ann_recall.std()))
print("eclf corss validation :  %0.4f precision with a standard deviation of %0.4f" % (accuracies_ann_precision.mean(), accuracies_ann_precision.std()))

eclf score               precision    recall  f1-score   support

           0     0.9932    0.9887    0.9909       882
           1     0.9730    0.9837    0.9783       367

    accuracy                         0.9872      1249
   macro avg     0.9831    0.9862    0.9846      1249
weighted avg     0.9873    0.9872    0.9872      1249



In [ ]:
#原版














print("this is naive_bayes part")
print("=======================================================================================================")
from sklearn.naive_bayes import MultinomialNB
nb= MultinomialNB()
nb.fit(X_train,y_train)
y_pred = (nb.predict(X_test) > 0.5).astype("int32")
from sklearn.metrics import classification_report
print("nb score",classification_report(y_test,y_pred, digits=4))


from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

from sklearn.model_selection import cross_val_score
nb2= MultinomialNB()
scores_nb2 = cross_val_score(nb2, X, y, cv=10)
print("nb corss validation :  %0.4f accuracy with a standard deviation of %0.4f" % (scores_nb2.mean(), scores_nb2.std()))
y_pred_nb = (nb.predict(X_test) > 0.5).astype("int32")
y_pred_nb_cv = cross_val_predict(nb2, X, y, cv=10)
print(classification_report(y_pred_nb_cv,y, digits=4))



print("this is svm part")
print("=======================================================================================================")

from sklearn import svm
svmc= svm.SVC()
svmc.fit(X_tfidf_train,y_tfidf_train)
y_pred = svmc.predict(X_tfidf_test)
from sklearn.metrics import classification_report
print("svmc score",classification_report(y_tfidf_test,y_pred, digits=4))
from sklearn.metrics import confusion_matrix
confusion_matrix(y_tfidf_test, y_pred)
from sklearn.model_selection import cross_val_score
svmc2= svm.SVC()
scores_svm = cross_val_score(svmc2, X_tfidf, y, cv=10)
print("svm corss validation :  %0.4f accuracy with a standard deviation of %0.4f" % (scores_svm.mean(), scores_svm.std()))
y_pred_svm = (svmc.predict(X_test) > 0.5).astype("int32")
y_pred_svm_cv = cross_val_predict(svmc2, X_tfidf, y, cv=10)
print(classification_report(y_pred_svm_cv,y, digits=4))




print("this is r-tree part")
print("=======================================================================================================")
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification


R_tree = RandomForestClassifier(n_estimators = 1000, min_samples_split = 10, max_depth=100, random_state = 42)
R_tree.fit(X_tfidf_train, y_tfidf_train)



y_pred = R_tree.predict(X_tfidf_test)
from sklearn.metrics import classification_report
print("R_tree score",classification_report(y_tfidf_test,y_pred, digits=4))


from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

from sklearn.model_selection import cross_val_score
R_tree2 = RandomForestClassifier(n_estimators = 1000, min_samples_split = 10, max_depth=100, random_state = 42)
scores_clf_model = cross_val_score(R_tree2, X_tfidf, y, cv=10)
print("R_tree corss validation :  %0.4f accuracy with a standard deviation of %0.4f" % (scores_clf_model.mean(), scores_clf_model.std()))
y_pred_tree = (R_tree.predict(X_test) > 0.5).astype("int32")
y_pred_tree_cv = cross_val_predict(R_tree2, X_tfidf, y, cv=10)
print(classification_report(y_pred_tree_cv,y, digits=4))









this is naive_bayes part
nb score               precision    recall  f1-score   support

           0     0.8979    1.0000    0.9462       897
           1     1.0000    0.7102    0.8306       352

    accuracy                         0.9183      1249
   macro avg     0.9489    0.8551    0.8884      1249
weighted avg     0.9267    0.9183    0.9136      1249

nb corss validation :  0.9808 accuracy with a standard deviation of 0.0055
              precision    recall  f1-score   support

           0     0.9836    0.9892    0.9864      3511
           1     0.9740    0.9609    0.9674      1483

    accuracy                         0.9808      4994
   macro avg     0.9788    0.9750    0.9769      4994
weighted avg     0.9807    0.9808    0.9807      4994

this is svm part
svmc score               precision    recall  f1-score   support

           0     0.9977    0.9795    0.9885       877
           1     0.9536    0.9946    0.9737       372

    accuracy                         0.9840  

In [ ]:






print("this is LSTM part")
print("=======================================================================================================")





message_bow2 = message_bow.copy()
raw_df2 = raw_df.copy()

texts1 = np.asarray(message_bow2)
labels1 = np.asarray(raw_df2['label'])



import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

import keras
from keras.models import Sequential
from keras.layers import Dense


from keras.layers import SimpleRNN, Embedding, Dense, LSTM
from keras.models import Sequential

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# number of words used as features
max_features = 500000
# cut off the words after seeing 500 words in each document(email)
maxlen = 500


# we will use 80% of data as training, 25% as validation data
training_samples = int(4994 * .75)
validation_samples = int(4994 - training_samples)
# sanity check
print(len(raw_df2['email']) == (training_samples + validation_samples))
print("The number of training {0}, validation {1} ".format(training_samples, validation_samples))

tokenizer = Tokenizer()
tokenizer.fit_on_texts(raw_df2['email'])
sequences = tokenizer.texts_to_sequences(raw_df2['email'])

word_index = tokenizer.word_index
print("Found {0} unique words: ".format(len(word_index)))

data = pad_sequences(sequences, maxlen=maxlen)

print("data shape: ", data.shape)

np.random.seed(42)
# shuffle data
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels1 = labels.copy()
labels1 = np.asarray(labels1)
labels1 = y[indices]


texts_train = data[:training_samples]
y_train_rnn = labels1[:training_samples]
texts_test = data[training_samples:]
y_test_rnn = labels1[training_samples:]






max_features =500000
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(Dropout(0.5))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

history_ltsm = model.fit(texts_train, y_train_rnn, epochs=30, batch_size=32)

pred = (model.predict(texts_test) > 0.5).astype("int32")
acc = model.evaluate(texts_test, y_test_rnn)
proba_ltsm = (model.predict(texts_test) > 0.5).astype("int32")
from sklearn.metrics import confusion_matrix
print("Test loss is {0:.4f} accuracy is {1:.4f}  ".format(acc[0],acc[1]))

from sklearn.metrics import classification_report
print("LSTM score",classification_report(y_test_rnn,pred, digits=4))
print(confusion_matrix(pred, y_test_rnn))










def create_keras_model():


    model2 = Sequential()
    model2.add(Embedding(max_features, 128))
    model2.add(Dropout(0.5))
    model2.add(LSTM(128, return_sequences=True))
    model2.add(Dropout(0.5))
    model2.add(LSTM(128))
    model2.add(Dropout(0.2))
    model2.add(Dense(1, activation='sigmoid'))
    model2.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])



    return model2


from keras.wrappers.scikit_learn import KerasClassifier

LSTMmodel = KerasClassifier(
        build_fn=create_keras_model, 
        batch_size=32, epochs=30)


LSTMmodel._estimator_type = "classifier"



from sklearn.model_selection import cross_val_score
accuracies_LSTM = cross_val_score(estimator=LSTMmodel, scoring="accuracy", X=data, y=labels1, cv=5)

print("LSTM corss validation :  %0.4f accuracy with a standard deviation of %0.4f" % (accuracies_LSTM.mean(), accuracies_LSTM.std()))











print("this is artifacial natual network part")
print("=======================================================================================================")

# Part 2 - Now let's make the ANN!


import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 50358))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))


# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
annn = classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set

ann = classifier.fit(X_train_ann, y_train, batch_size = 5, epochs = 12)


# Predicting the Test set results
y_pred = (classifier.predict(X_test) > 0.5).astype("int32")
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix

classifier.summary()

from sklearn.metrics import classification_report
print("ann score",classification_report(y_test,y_pred, digits=4))
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)








def create_keras_model():

    model = Sequential()

    # Adding the input layer and the first hidden layer
    model.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 50358))

    # Adding the second hidden layer
    model.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))


    # Adding the output layer
    model.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

    # Compiling the ANN
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

    # Fitting the ANN to the Training set



    # Predicting the Test set results

    return model


from keras.wrappers.scikit_learn import KerasClassifier

ann2 = KerasClassifier(
        build_fn=create_keras_model, 
        batch_size=5, epochs=12)


ann2._estimator_type = "classifier"


accuracies_ann2 = cross_val_score(estimator=ann2, scoring="accuracy", 
    X=X_ann, y=y, cv=5)

print("ann corss validation :  %0.4f accuracy with a standard deviation of %0.4f" % (accuracies_ann2.mean(), accuracies_ann2.std()))












this is naive_bayes part
nb score               precision    recall  f1-score   support

           0     0.8981    1.0000    0.9463       890
           1     1.0000    0.7187    0.8363       359

    accuracy                         0.9191      1249
   macro avg     0.9490    0.8593    0.8913      1249
weighted avg     0.9274    0.9191    0.9147      1249

nb corss validation :  0.9814 accuracy with a standard deviation of 0.0059
this is svm part
svmc score               precision    recall  f1-score   support

           0     0.9966    0.9788    0.9876       896
           1     0.9485    0.9915    0.9695       353

    accuracy                         0.9824      1249
   macro avg     0.9726    0.9851    0.9786      1249
weighted avg     0.9830    0.9824    0.9825      1249

svm corss validation :  0.9874 accuracy with a standard deviation of 0.0031
this is r-tree part
R_tree score               precision    recall  f1-score   support

           0     0.9943    0.9710    0.9825  

In [ ]:
#测试ann svm lstm
def create_keras_model():


    model2 = Sequential()
    model2.add(Embedding(max_features, 128))
    model2.add(Dropout(0.5))
    model2.add(LSTM(128, return_sequences=True))
    model2.add(Dropout(0.5))
    model2.add(LSTM(128))
    model2.add(Dropout(0.2))
    model2.add(Dense(1, activation='sigmoid'))
    model2.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])



    return model2


from keras.wrappers.scikit_learn import KerasClassifier

LSTMmodel = KerasClassifier(
        build_fn=create_keras_model, 
        batch_size=128, epochs=500)


LSTMmodel._estimator_type = "classifier"


print("this is 3 voting part")
print("=======================================================================================================")


#voting method for no cross validation
counter = 0
y_pred1 = y_pred = (classifier.predict(X_test) > 0.5).astype("int32")
y_pred1 = (y_pred > 0.5)
y_pred2 = (model.predict(texts_test) > 0.5).astype("int32")
y_pred3 = y_pred_svm
y_predt = []
for i in range(len(y_pred1)):
    if y_pred1[i] == True:
        counter += 1
    if y_pred2[i] == True:
        counter += 1    
    if y_pred3[i] == True:
        counter += 1
    if counter>=2:
        y_predt.append(1)
    if counter<2:
        y_predt.append(0)
    counter = 0
    
from sklearn.metrics import classification_report
print("voting score",classification_report(y_test,y_predt, digits=4))
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_predt)  
    
    
#voting method for cross validation



#cossvalidation for ann2
y_pred_ann_cv = cross_val_predict(ann2, X_ann, y, cv=5)
y_pred_lstm_cv = cross_val_predict(LSTMmodel, data, y, cv=5)


counter = 0
y_pred1_cv = y_pred_ann_cv

y_pred2_cv = y_pred_lstm_cv
y_pred3_cv = y_pred_svm_cv
y_predt_cv = []
for i in range(len(y_pred1_cv)):
    if y_pred1_cv[i] == True:
        counter += 1
    if y_pred2_cv[i] == True:
        counter += 1    
    if y_pred3_cv[i] == True:
        counter += 1
    if counter>=2:
        y_predt_cv.append(1)
    if counter<2:
        y_predt_cv.append(0)
    counter = 0
    
from sklearn.metrics import classification_report
print("v3 voting score(cross validation )")
print(classification_report(y,y_predt_cv, digits=4))

from sklearn.metrics import accuracy_score
score_voting_vc = accuracy_score(y, y_predt_cv)
print("3 voting corss validation（SVM/ANN/LSTM） :  %0.4f accuracy " % (score_voting_vc.mean()))


from sklearn.metrics import confusion_matrix
confusion_matrix(y, y_predt_cv)  




流式输出内容被截断，只能显示最后 5000 行内容。
32/32 [==============================] - 3s 87ms/step - loss: 0.3071 - acc: 0.8706
Epoch 7/500
32/32 [==============================] - 3s 87ms/step - loss: 0.2896 - acc: 0.8793
Epoch 8/500
32/32 [==============================] - 3s 87ms/step - loss: 0.2557 - acc: 0.8899
Epoch 9/500
32/32 [==============================] - 3s 87ms/step - loss: 0.2457 - acc: 0.8934
Epoch 10/500
32/32 [==============================] - 3s 87ms/step - loss: 0.2328 - acc: 0.9016
Epoch 11/500
32/32 [==============================] - 3s 87ms/step - loss: 0.2208 - acc: 0.9009
Epoch 12/500
32/32 [==============================] - 3s 87ms/step - loss: 0.2092 - acc: 0.9109
Epoch 13/500
32/32 [==============================] - 3s 87ms/step - loss: 0.2007 - acc: 0.9134
Epoch 14/500
32/32 [==============================] - 3s 87ms/step - loss: 0.1923 - acc: 0.9139
Epoch 15/500
32/32 [==============================] - 3s 88ms/step - loss: 0.1865 - acc: 0.9146
Epoch 16/500
32/32 [=========

array([[3494,   37],
       [  28, 1435]])

In [ ]:



print("this is 3 voting part")
print("=======================================================================================================")


#voting method for no cross validation
counter = 0
y_pred1 = y_pred = (classifier.predict(X_test) > 0.5).astype("int32")
y_pred1 = (y_pred > 0.5)
y_pred2 = y_pred_nb
y_pred3 = y_pred_svm
y_predt = []
for i in range(len(y_pred1)):
    if y_pred1[i] == True:
        counter += 1
    if y_pred2[i] == True:
        counter += 1    
    if y_pred3[i] == True:
        counter += 1
    if counter>=2:
        y_predt.append(1)
    if counter<2:
        y_predt.append(0)
    counter = 0
    
from sklearn.metrics import classification_report
print("voting score",classification_report(y_test,y_predt, digits=4))
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_predt)  
    
    
#voting method for cross validation



#cossvalidation for ann2
y_pred_ann_cv = cross_val_predict(ann2, X_ann, y, cv=5)


counter = 0
y_pred1_cv = y_pred_ann_cv

y_pred2_cv = y_pred_nb_cv
y_pred3_cv = y_pred_svm_cv
y_predt_cv = []
for i in range(len(y_pred1_cv)):
    if y_pred1_cv[i] == True:
        counter += 1
    if y_pred2_cv[i] == True:
        counter += 1    
    if y_pred3_cv[i] == True:
        counter += 1
    if counter>=2:
        y_predt_cv.append(1)
    if counter<2:
        y_predt_cv.append(0)
    counter = 0
    
from sklearn.metrics import classification_report
print("v3 voting score(cross validation )")
print(classification_report(y,y_predt_cv, digits=4))

from sklearn.metrics import accuracy_score
score_voting_vc = accuracy_score(y, y_predt_cv)
print("3 voting corss validation :  %0.4f accuracy " % (score_voting_vc.mean()))


from sklearn.metrics import confusion_matrix
confusion_matrix(y, y_predt_cv)  







print("this is 5 voting part")
print("=======================================================================================================")


#voting method for no cross validation
counter = 0
y_pred11 = y_pred = (classifier.predict(X_test) > 0.5).astype("int32")
y_pred11 = (y_pred > 0.5)
y_pred22 = y_pred_nb
y_pred33 = y_pred_svm
y_pred44 = y_pred_tree
y_pred55 = (model.predict(texts_test) > 0.5).astype("int32")
y_pred55 = (y_pred55 > 0.5)
y_predt = []
for i in range(len(y_pred1)):
    if y_pred11[i] == True:
        counter += 1
    if y_pred22[i] == True:
        counter += 1    
    if y_pred33[i] == True:
        counter += 1
    if y_pred44[i] == True:
        counter += 1
    if y_pred55[i] == True:
        counter += 1
    if counter>=3:
        y_predt.append(1)
    if counter<3:
        y_predt.append(0)
    counter = 0
    
from sklearn.metrics import classification_report
print("voting score",classification_report(y_test,y_predt, digits=4))
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_predt)  
    
    
#voting method for cross validation

from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
X_sc = sc.fit(X)
X_sc2 = X_sc.transform(X)

#cossvalidation for ann2


y_pred_lstm_cv = cross_val_predict(LSTMmodel, data, y, cv=5)

counter = 0
y_pred1_cv = y_pred_ann_cv

y_pred2_cv = y_pred_nb_cv
y_pred3_cv = y_pred_svm_cv
y_pred4_cv = y_pred_tree_cv
y_pred5_cv = y_pred_lstm_cv

y_predt_cv = []
for i in range(len(y_pred1_cv)):
    if y_pred1_cv[i] == True:
        counter += 1
    if y_pred2_cv[i] == True:
        counter += 1    
    if y_pred3_cv[i] == True:
        counter += 1
    if y_pred4_cv[i] == True:
        counter += 1
    if y_pred5_cv[i] == True:
        counter += 1
    if counter>=3:
        y_predt_cv.append(1)
    if counter<3:
        y_predt_cv.append(0)
    counter = 0
    
from sklearn.metrics import classification_report
print("5 voting score(cross validation )")
print(classification_report(y,y_predt_cv, digits=4))

from sklearn.metrics import accuracy_score
score_voting_vc = accuracy_score(y, y_predt_cv)
print("5 voting corss validation :  %0.4f accuracy " % (score_voting_vc.mean()))


from sklearn.metrics import confusion_matrix
confusion_matrix(y, y_predt_cv)  






















print("------------------------------------------------------------------------------------")
print("花费时间：  %s seconds " % (time.time() - start_time))

this is 3 voting part
voting score               precision    recall  f1-score   support

           0     0.9965    0.9493    0.9723       888
           1     0.8883    0.9917    0.9372       361

    accuracy                         0.9616      1249
   macro avg     0.9424    0.9705    0.9547      1249
weighted avg     0.9652    0.9616    0.9622      1249

Epoch 1/12
799/799 [==============================] - 2s 3ms/step - loss: 0.2398 - accuracy: 0.9194
Epoch 2/12
799/799 [==============================] - 2s 2ms/step - loss: 0.0288 - accuracy: 0.9942
Epoch 3/12
799/799 [==============================] - 2s 2ms/step - loss: 0.0110 - accuracy: 0.9980
Epoch 4/12
799/799 [==============================] - 2s 2ms/step - loss: 0.0056 - accuracy: 0.9995
Epoch 5/12
799/799 [==============================] - 2s 2ms/step - loss: 0.0033 - accuracy: 0.9997
Epoch 6/12
799/799 [==============================] - 2s 2ms/step - loss: 0.0019 - accuracy: 0.9997
Epoch 7/12
799/799 [==================

InternalError: ignored